<h3>This notebook is used to access solutions from pdpt problem</h3>

In [1]:

import math, os, sys
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022'
sys.path.insert(1, src_dir_)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from src.util import read_pickle, print_dict, read_coordinates_FastMap, manual_stop
from src.pdotw_mip import eval_pdotw_sol
from matplotlib.lines import Line2D
import matplotlib.font_manager as font_manager

DATA_DIR = "/home/tan/Documents/GitHub/pdpt_2022"


In [ ]:
from socket import AF_UNIX


def plot_ini_sol(dir_, case_num,  pdpt_ins, pdotw_sol, ):
    ms = 5
    radius = math.sqrt(ms)/2.
    legend_font = font_manager.FontProperties(family='serif',style='normal', size=12)

    node_list = pdpt_ins['nodes']
    truck_list = pdpt_ins['truck']
    truck_route = pdotw_sol['route']['truck_route']
    num_truck = len(truck_route)
    coor_fastmap_filename = os.path.join(dir_, f'data/raw/case{case_num}', 'coordinates_FastMap.txt')
    node_coor = read_coordinates_FastMap(coor_fastmap_filename, node_list, 0)
    # coor_fastmap = pdpt_ins['coordinate']


    # figures = []
    cmap = plt.get_cmap('gist_rainbow', len(truck_route))

    truck_idx = 0
    for truck_key, route_ in truck_route.items():
        if len(route_) > 1:
            fig, ax = plt.subplots(1, 1, figsize=(20, 10))
            ax.plot(*np.array(list(node_coor.values())).T, 'o',  ms=ms, mfc='None', mec='k', mew=2)
            for i in range(len(route_)-1):
                node_curr, node_next = route_[i], route_[i+1]
                x0, y0 = node_coor[node_curr]
                # dx, dy = (np.array(node_coor[node_next])-np.array(node_coor[node_curr]))
                x1, y1 = node_coor[node_next]
                # ax1.arrow(x0, y0, dx, dy, color=truck_colors[truck_idx], 
                #           width = 1, head_width=4, head_length=2,
                #           length_includes_head = True, linewidth=2, linestyle='-',
                #           alpha=0.3+truck_idx*0.1)
                arrow = mpl.patches.FancyArrowPatch(posA=(x0,y0), posB=(x1,y1), 
                                        arrowstyle='-|>', mutation_scale=20, 
                                        shrinkA=radius, shrinkB=radius*2,
                                        color = cmap(truck_idx/num_truck), 
                                        # alpha=0.2+truck_idx*0.1, 
                                        linewidth=1.5,)
                ax.add_patch(arrow)
            source, dest = truck_list[truck_key][0], truck_list[truck_key][1]
            ax.plot(*node_coor[source], 'p', mec = cmap(truck_idx/num_truck),
                    mfc='None', ms=ms*1.5, alpha =0.6, lw=2)
            ax.plot(*node_coor[dest], 's', mec = cmap(truck_idx/num_truck),
                    mfc='None', ms=ms*1.5, alpha =0.6, lw=2)
            truck_idx += 1

            legend_elements = []
            legend_elements.append(Line2D([0], [0],  marker='p', color='w', label='truck_origin',
                            markerfacecolor='None', mec='k',  markersize=ms*4))
            legend_elements.append(Line2D([0], [0],  marker='s', color='w', label='truck_dest',
                                markerfacecolor='None', mec='k',  markersize=ms*4))

            ax.set_title(f'PDOTW solution: Truck route for [{truck_key}]', size=20, font='serif')
            ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=4, prop = legend_font)

            rec = [plt.Rectangle(
                    # (lower-left corner), width, height
                    (0.02, 0.02), 0.97, 0.97, fill=False, color="k", lw=2, 
                    zorder=1000, transform=fig.transFigure, figure=fig)]
            fig.patches.extend(rec)

            ax.axis('off')
            ax.set_aspect('equal')
            plt.ioff()
            fig.savefig(os.path.join(dir_, f'out/', f'case{case_num}_initSol_{truck_key}.png'), dpi=150)
            plt.close()
            # figures.append([fig, ax])
    # return figures

def read_pdotw_sol(dir_, case_num):

    pdpt_ins_filename = os.path.join(DATA_DIR, f'data/case{case_num}.pkl')
    pdpt_ini_sol_res_filename = os.path.join(DATA_DIR, f'out/case{case_num}_initSol.pkl')

    pdpt_ins = read_pickle(pdpt_ins_filename)
    pdotw_sol = read_pickle(pdpt_ini_sol_res_filename)
    

    figures = plot_ini_sol(dir_, case_num, pdpt_ins, pdotw_sol)

        




    # truck_route = pdpt_ini_sol['route']['truck_route']
    # cargo_route = pdpt_ini_sol['route']['cargo_route']

    # print_dict(truck_route)
    # for truck_key, truck_route_ in truck_route.items():
    #     if len(truck_route_) > 1:
    # print()
    # # print_dict(cargo_route)



In [ ]:
# case_num = 1

# for case_num in range(5,6,1):

#     read_pdotw_sol(DATA_DIR, case_num)

<hr style="height:10px;">
<hr style="height:10px;">

def collect_res(dir_):

In [5]:
def collect_res(dir_, case_num):
    pdpt_ins_filename = os.path.join(DATA_DIR, f'data/case{case_num}.pkl')
    pdotw_sol_filename = os.path.join(DATA_DIR, f'out/case{case_num}_initSol.pkl')
    pdpt_sol_filename = os.path.join(DATA_DIR, f'out/case{case_num}_rasdSol.pkl')

    pdpt_ins = read_pickle(pdpt_ins_filename)
    constant = pdpt_ins['constant']
    edge_shortest = pdpt_ins['edge_shortest']
    cargo_list = pdpt_ins['cargo']


    pdotw_sol = read_pickle(pdotw_sol_filename)
    pdotw_used_trucks = pdotw_sol['route']['used_truck'] 
    pdotw_used_truck_route = pdotw_sol['route']['truck_route'] 
    pdotw_cargo_route = pdotw_sol['route']['cargo_route']
    pdotw_costs = list(pdotw_sol['cost'].values())

    undelivered_cargos = [cargo_key for cargo_key in cargo_list.keys() if len(pdotw_cargo_route[cargo_key])==0]

    # pdpt_sol = read_pickle(pdpt_sol_filename)

    # # print(pdotw_sol['route'])
    # trucks_ = pdpt_sol['route']['truck_route'].keys()


    # for t_key in trucks_:
    #     pdotw_used_trucks.remove(t_key)
    #     del pdotw_used_truck_route[t_key]

    

    truck_cost_, travel_cost_ = eval_pdotw_sol(constant, edge_shortest, pdotw_used_trucks, pdotw_used_truck_route)
    cost_rest_routes = [truck_cost_, travel_cost_, 0]
    # rasd_costs_ = list(pdpt_sol['cost'].values())
    # rasd_costs = [ rasd_costs_[i] + cost_rest_routes[i] for i in range(len(rasd_costs_))]
    # rasd_costs.append(sum(rasd_costs))

    # phase = ['PDOTW', 'RASD (1 iter)']
    # data = [pdotw_costs, rasd_costs]
    phase = ['PDOTW']
    data = [len(undelivered_cargos), *pdotw_costs, sum(pdotw_costs), round(sum(pdotw_sol['MIP']['runtime']),2)]

    cost_name = list(pdotw_sol['cost'].keys())
    cost_name.insert(0, 'undel. parcels')
    cost_name.append('total cost')
    cost_name.append('runtime')

    row_format ="{:>10}" + "{:>15}" * (len(cost_name))

    print(row_format.format("", *cost_name))
    print(row_format.format(phase[0], *data))

    # row_format ="{:>15}" * (len(cost_name) + 1)
    # print(row_format.format("", *cost_name))

    # for team, row in zip(phase, data):
    #     print(row_format.format(team, *row))

for case_num in range(1,6, 1):
# for case_num in [1,]:
    print(f'preliminary results on PDPT case {case_num}')
    collect_res(DATA_DIR, case_num)


preliminary results on PDPT case 1
[2, 630000.0, 545750.0, 1175750.0, 315.06]
           undel. parcels     truck_cost    travel_cost     total cost        runtime
     PDOTW              2       630000.0       545750.0      1175750.0         315.06
preliminary results on PDPT case 2
[0, 450000.0, 422650.0, 872650.0, 210.15]
           undel. parcels     truck_cost    travel_cost     total cost        runtime
     PDOTW              0       450000.0       422650.0       872650.0         210.15
preliminary results on PDPT case 3
[0, 420000.0, 432100.0, 852100.0, 157.48]
           undel. parcels     truck_cost    travel_cost     total cost        runtime
     PDOTW              0       420000.0       432100.0       852100.0         157.48
preliminary results on PDPT case 4
[0, 480000.0, 429900.0, 909900.0, 235.76]
           undel. parcels     truck_cost    travel_cost     total cost        runtime
     PDOTW              0       480000.0       429900.0       909900.0         235.76
pre

<hr style="height:10px;">
<hr style="height:10px;">